## PyTorch autograd examples

In this example, we calculate transfer maps using PyTorch autograd Jacobian and compare times against numerical differentiation. We also calculate the gradient and the Hessian of the beamsize of a 10 quadrupole lattice with respect to the quadrupole strengths. 

In [1]:
import torch
import numpy as np

from bmadx import Particle, Drift, Quadrupole
from bmadx import track_element, track_lattice
from bmadx import M_ELECTRON


import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' # for high DPI displays in MacOS

torch.set_printoptions(precision= 8, sci_mode=True)

In [2]:
# Create torch particle beam
s = 0.0 # initial s
p0c = 4e7 # reference momentum in eV

# Initial beam distribution
n_particles = 10_000 # number of particles in beam
coords = np.random.multivariate_normal(mean = np.zeros(6),
                                       cov = 1e-6*np.identity(6),
                                       size = n_particles
                                      )
beam_np = Particle(*coords.T, s=s, p0c=p0c, mc2=M_ELECTRON)

beam_torch = Particle(*torch.tensor(coords.T),
                      s=torch.tensor(s),
                      p0c=torch.tensor(p0c),
                      mc2=torch.tensor(M_ELECTRON))

In [3]:
# Tracking though element takes incoming particle coordinate
# type (i.e., torch tensor in this case) for the tracking regarldless
# the element parameter types.
q = Quadrupole(L=0.1, K1=10.0)
track_element(beam_torch, q)

Particle(x=tensor([ 8.57547392e-04, -4.48009639e-05, -9.48887744e-04,  ...,
         1.46583808e-03,  4.33444682e-05,  8.17708981e-04],
       dtype=torch.float64), px=tensor([-1.21091155e-03, -1.07366598e-03, 3.28958473e-04,  ..., -1.73374331e-03,
        1.09766809e-04, -6.71714216e-04], dtype=torch.float64), y=tensor([-5.84690261e-04, -2.26083872e-03,  1.06809707e-03,  ...,
         9.79335079e-05, -5.47009867e-05,  4.31609461e-04],
       dtype=torch.float64), py=tensor([-1.60181842e-03, -4.19318714e-03,  5.88492562e-04,  ...,
         6.54512373e-04,  4.30206959e-04, -8.72212250e-05],
       dtype=torch.float64), z=tensor([-7.75706244e-04,  6.02419314e-04, -1.52169083e-03,  ...,
        -1.47490889e-03,  1.48174210e-03, -6.26448989e-05],
       dtype=torch.float64), pz=tensor([ 2.33228564e-04, -1.92086018e-03,  1.25268022e-05,  ...,
         1.22779163e-03, -7.36382790e-04, -3.44036896e-04],
       dtype=torch.float64), s=tensor(1.00000000e-01, dtype=torch.float64), p0c=tensor(4.0

In [4]:
# we can use PyTorch autograd to calculate derivatives

# calculate Jacobian around 0 through quadrupole:
# define tracking as function of 6d coordinates:
f_quadrupole_torch = lambda coord: track_element(Particle(*coord,
                                                    s = torch.tensor(s),
                                                    p0c = torch.tensor(p0c),
                                                    mc2 = torch.tensor(M_ELECTRON)), 
                                                 q)[:6]
f_quadrupole_np = lambda coord: np.array(track_element(Particle(*coord,
                                                                s = s,
                                                                p0c = p0c,
                                                                mc2 = M_ELECTRON), 
                                                       q)[:6])

In [5]:
import torch.autograd.functional as ad

In [6]:
%%time
# Jacobian evaluated at 0 using pytorch autograd:
J_ad = ad.jacobian(f_quadrupole_torch, torch.zeros(6))
J_ad = torch.vstack(J_ad)
J_ad

CPU times: user 5.59 ms, sys: 4.79 ms, total: 10.4 ms
Wall time: 16.8 ms


tensor([[ 9.50415254e-01,  9.83416438e-02,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [-9.83416438e-01,  9.50415254e-01, -0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.05041802e+00,  1.01675019e-01,
          0.00000000e+00, -0.00000000e+00],
        [-0.00000000e+00,  0.00000000e+00,  1.01675022e+00,  1.05041802e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.63173318e-05],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00]])

In [7]:
import numdifftools as nd

In [8]:
%%time
# Jacobian evaluated at 0 using numerical differentiation:
J_nd = nd.Jacobian(f_quadrupole_np)(np.zeros(6))
J_nd

CPU times: user 8.91 ms, sys: 1.33 ms, total: 10.2 ms
Wall time: 10.8 ms


/Users/jpga/Repositories/Bmad-X/bmadx/bmad/low_level/track_a_quadrupole.py:40: RuntimeWarning: divide by zero encountered in double_scalars
  k1 = b1/(l*rel_p)
/Users/jpga/Repositories/Bmad-X/bmadx/bmad/low_level/quad_mat2_calc.py:28: RuntimeWarning: invalid value encountered in cos
  cx = cos(sk_l) * (k1<=0) + cosh(sk_l) * (k1>0)
/Users/jpga/Repositories/Bmad-X/bmadx/bmad/low_level/quad_mat2_calc.py:28: RuntimeWarning: invalid value encountered in double_scalars
  cx = cos(sk_l) * (k1<=0) + cosh(sk_l) * (k1>0)
/Users/jpga/Repositories/Bmad-X/bmadx/bmad/low_level/quad_mat2_calc.py:29: RuntimeWarning: invalid value encountered in sin
  sx = (sin(sk_l)/(sqrt_k))*(k1<=0) + (sinh(sk_l)/(sqrt_k))*(k1>0)
/Users/jpga/Repositories/Bmad-X/bmadx/bmad/low_level/quad_mat2_calc.py:29: RuntimeWarning: invalid value encountered in double_scalars
  sx = (sin(sk_l)/(sqrt_k))*(k1<=0) + (sinh(sk_l)/(sqrt_k))*(k1>0)


array([[ 9.50415280e-01,  9.83416469e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.83416469e-01,  9.50415280e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.05041806e+00,
         1.01675020e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.01675020e+00,
         1.05041806e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.63173324e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [9]:
# calculate gradient of beamsize with respect to quadrupole strengths:

# fixed parameters:
half_drift = Drift(L=0.9/2) # half drift
L_q = 0.1 # quad length

def beamsize(beam_in, quad_strengths):
    """Function of beamsize with respect to quad strengths"""
    # create lattice of len(quad_strengths) quadrupoles separated by drifts:
    lattice = [] # initialize lattice
    for k1 in quad_strengths:
        lattice.append( half_drift )
        lattice.append( Quadrupole(L=L_q, K1=k1) )
        lattice.append( half_drift )
        
    beam_out = track_lattice(beam_in, lattice)
    return beam_out.x.std()

beamsize_torch = lambda k1s: beamsize(beam_torch, k1s)
beamsize_np = lambda k1s: beamsize(beam_np, k1s)

In [10]:
%%time
# calculate gradient of beamsize with respect to quad strengths using torch autograd:
grad_ad = ad.jacobian(beamsize_torch, torch.zeros(10))
grad_ad

CPU times: user 26 ms, sys: 10.5 ms, total: 36.5 ms
Wall time: 33.2 ms


tensor([-5.03742893e-04, -1.27981242e-03, -1.85958017e-03, -2.24304572e-03, -2.43020966e-03,
        -2.42107152e-03, -2.21563131e-03, -1.81388925e-03, -1.21584523e-03, -4.21499339e-04])

In [11]:
%%time
# calculate gradient of beamsize with respect to quad strengths using numerical diff:
grad_nd = nd.Jacobian(beamsize_np)(np.zeros(10))
grad_nd

CPU times: user 2.82 s, sys: 211 ms, total: 3.03 s
Wall time: 3.03 s


array([[-0.00055292, -0.00132895, -0.00190869, -0.00229214, -0.00247929,
        -0.00247015, -0.00226472, -0.001863  , -0.00126499, -0.00047068]])

In [12]:
%%time
# calculate Hessian of beamsize with respect to quad strengths using torch autograd:
hes_ad = ad.hessian(beamsize_torch, torch.zeros(10))
hes_ad

CPU times: user 320 ms, sys: 64.8 ms, total: 385 ms
Wall time: 373 ms


tensor([[8.21035646e-05, 1.06269261e-04, 1.26630301e-04, 1.38149117e-04,
         1.40825709e-04, 1.34660077e-04, 1.19652228e-04, 9.58021556e-05,
         6.31098592e-05, 2.15753371e-05],
        [1.06269261e-04, 5.26204931e-05, 1.42522898e-04, 2.16387867e-04,
         2.61417008e-04, 2.77610321e-04, 2.64967792e-04, 2.23489449e-04,
         1.53175279e-04, 5.40252731e-05],
        [1.26630301e-04, 1.42522898e-04, 3.18977691e-05, 1.68861079e-04,
         2.75631406e-04, 3.33612639e-04, 3.42804764e-04, 3.03207751e-04,
         2.14821644e-04, 7.76464294e-05],
        [1.38149117e-04, 2.16387867e-04, 1.68861079e-04, 1.79995786e-05,
         1.83468917e-04, 3.02667031e-04, 3.53163108e-04, 3.34957062e-04,
         2.48048978e-04, 9.24388078e-05],
        [1.40825709e-04, 2.61417008e-04, 2.75631406e-04, 1.83468917e-04,
         9.23205062e-06, 1.84773540e-04, 2.96042825e-04, 3.18737410e-04,
         2.52857251e-04, 9.84024009e-05],
        [1.34660077e-04, 2.77610321e-04, 3.33612639e-04, 3.0

In [13]:
%%time
# calculate Hessian of beamsize with respect to quad strengths using numerical diff:
hes_nd = nd.Hessian(beamsize_np)(np.zeros(10))
hes_nd

CPU times: user 28.3 s, sys: 3.52 s, total: 31.9 s
Wall time: 31.9 s


array([[8.38666115e-05, 1.15137218e-04, 1.38428177e-04, 1.51895938e-04,
        1.55540500e-04, 1.49361863e-04, 1.33360026e-04, 1.07534991e-04,
        7.18867567e-05, 2.64153234e-05],
       [1.15137218e-04, 5.69706696e-05, 1.58166901e-04, 2.33977690e-04,
        2.79972674e-04, 2.96151854e-04, 2.82515228e-04, 2.39062798e-04,
        1.65794563e-04, 6.27105235e-05],
       [1.38428177e-04, 1.58166901e-04, 3.81804346e-05, 1.89318768e-04,
        2.97051863e-04, 3.55016867e-04, 3.63213781e-04, 3.21642604e-04,
        2.30303336e-04, 8.91959771e-05],
       [1.51895938e-04, 2.33977690e-04, 1.89318768e-04, 2.55601974e-05,
        2.06778066e-04, 3.25956904e-04, 3.75455685e-04, 3.55274409e-04,
        2.65413075e-04, 1.05871684e-04],
       [1.55540500e-04, 2.79972674e-04, 2.97051863e-04, 2.06778066e-04,
        1.74161665e-05, 2.08971964e-04, 3.19240940e-04, 3.39958213e-04,
        2.71123781e-04, 1.12737645e-04],
       [1.49361863e-04, 2.96151854e-04, 3.55016867e-04, 3.25956904e-04,
   

### fig, axs = plt.subplots(2, figsize=(10,10))
cm = axs[0].imshow(hes_ad.detach().numpy())
axs[0].set_xlabel(r'$k_i$')
axs[0].set_ylabel(r'$k_j$')
fig.colorbar(cm,ax=axs[0], label=r'$\partial^2 \sigma_x / (\partial k_i \partial k_j)$')
cm = axs[1].imshow(hes_nd)
axs[1].set_xlabel(r'$k_i$')
axs[1].set_ylabel(r'$k_j$')
fig.colorbar(cm,ax=axs[1], label=r'$\partial^2 \sigma_x / (\partial k_i \partial k_j)$')